<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV

In [3]:
working_dir = os.getcwd()
data_path = working_dir + '/data/'
processed_path = data_path + 'processed/'
model_path = working_dir + '/models/'
ensemble_path = model_path + 'ensemble/'

In [4]:
#load our training and testing data
X = pd.read_csv(processed_path + 'X.csv', index_col='Id')
y = np.load(processed_path + 'y.npy')
testing_set = pd.read_csv(processed_path + "testing_set.csv", index_col='Id')

In [5]:
#loading our preprocess pipeline
with open(model_path + 'preprocess_pipeline.h5', 'rb') as f:
    preprocessor = pickle.load(f)

In [6]:
#load our RandomForestRegressor 
with open(model_path + 'random_forest/best_RandomForest.pickle', 'rb') as f:
    rf_pipeline = pickle.load(f)
rf_regressor = rf_pipeline.named_steps['model']

In [7]:
#load our XGBoostRegressor
with open(model_path + 'XGBoost/best_XGBoost.pickle', 'rb') as f:
    xgb_regressor = pickle.load(f)

In [8]:
#load our CatBoostRegressor
with open(model_path + 'CatBoost/best_CatBoost.pickle', 'rb') as f:
    catboost_regressor = pickle.load(f)

In [9]:
X = preprocessor.fit_transform(X)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8,test_size = 0.2, random_state = 1)

In [10]:
X_train.shape

(1168, 310)

In [11]:
y_train.shape

(1168,)

In [12]:
rf_regressor

RandomForestRegressor(max_depth=40, min_samples_leaf=2, min_samples_split=5,
                      n_estimators=1600, random_state=42)

In [13]:
#define estimators for ensemble model
estimators = [
        ('rf_model', rf_regressor),
        ('xgb_model', xgb_regressor),
        ('cat_model', catboost_regressor),
        ('linear_model',LinearRegression())]

In [15]:
#define stacking regressor to create an ensemble model
stack = StackingRegressor(estimators=estimators,
                            final_estimator=RidgeCV(), cv=5,
                            passthrough = False,verbose = 2,
                            n_jobs=-1)

In [16]:
stack

StackingRegressor(cv=5,
                  estimators=[('rf_model',
                               RandomForestRegressor(max_depth=40,
                                                     min_samples_leaf=2,
                                                     min_samples_split=5,
                                                     n_estimators=1600,
                                                     random_state=42)),
                              ('xgb_model',
                               XGBRegressor(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0.4999379628659259,
                                            enable_categorical=False,
                                            gamma=0.0015642079516800683,
                                            gpu_id=...
                                            p

In [17]:
#fit stacking regressor
stack.fit(X_train,y_train)

0:	learn: 61520.6118474	total: 89.6ms	remaining: 1m 56s
1:	learn: 49263.2975674	total: 94.7ms	remaining: 1m 1s
2:	learn: 41565.8813804	total: 100ms	remaining: 43.4s
3:	learn: 36138.0943243	total: 105ms	remaining: 34.1s
4:	learn: 32140.9205237	total: 110ms	remaining: 28.5s
5:	learn: 29931.5639389	total: 115ms	remaining: 24.8s
6:	learn: 27930.1435458	total: 120ms	remaining: 22.2s
7:	learn: 26385.8550900	total: 125ms	remaining: 20.3s
8:	learn: 26033.8753486	total: 130ms	remaining: 18.6s
9:	learn: 25232.7111908	total: 135ms	remaining: 17.4s
10:	learn: 24963.2772580	total: 139ms	remaining: 16.3s
11:	learn: 24688.5863166	total: 144ms	remaining: 15.4s
12:	learn: 23787.7613188	total: 148ms	remaining: 14.7s
13:	learn: 23104.9198739	total: 156ms	remaining: 14.3s
14:	learn: 22304.4484585	total: 160ms	remaining: 13.7s
15:	learn: 22169.8145297	total: 166ms	remaining: 13.4s
16:	learn: 21978.1856325	total: 171ms	remaining: 12.9s
17:	learn: 21276.7096015	total: 176ms	remaining: 12.6s
18:	learn: 20866.

298:	learn: 1843.7817879	total: 1.72s	remaining: 5.75s
299:	learn: 1842.3846732	total: 1.72s	remaining: 5.74s
300:	learn: 1811.6190115	total: 1.73s	remaining: 5.73s
301:	learn: 1811.1679181	total: 1.73s	remaining: 5.73s
302:	learn: 1801.7963308	total: 1.74s	remaining: 5.72s
303:	learn: 1781.0255021	total: 1.74s	remaining: 5.71s
304:	learn: 1767.2924987	total: 1.75s	remaining: 5.71s
305:	learn: 1765.9755966	total: 1.75s	remaining: 5.7s
306:	learn: 1745.8731894	total: 1.76s	remaining: 5.69s
307:	learn: 1729.7125704	total: 1.76s	remaining: 5.69s
308:	learn: 1715.0760514	total: 1.77s	remaining: 5.68s
309:	learn: 1700.5603357	total: 1.78s	remaining: 5.67s
310:	learn: 1687.7583706	total: 1.78s	remaining: 5.67s
311:	learn: 1684.4807433	total: 1.79s	remaining: 5.66s
312:	learn: 1681.2684302	total: 1.79s	remaining: 5.65s
313:	learn: 1664.8644874	total: 1.8s	remaining: 5.65s
314:	learn: 1646.7038088	total: 1.8s	remaining: 5.64s
315:	learn: 1644.4262206	total: 1.81s	remaining: 5.64s
316:	learn: 1

600:	learn: 317.8157580	total: 3.58s	remaining: 4.17s
601:	learn: 316.0004911	total: 3.59s	remaining: 4.16s
602:	learn: 314.2258814	total: 3.59s	remaining: 4.15s
603:	learn: 311.0626930	total: 3.6s	remaining: 4.15s
604:	learn: 309.1738583	total: 3.6s	remaining: 4.14s
605:	learn: 307.6954368	total: 3.61s	remaining: 4.13s
606:	learn: 306.3775662	total: 3.62s	remaining: 4.13s
607:	learn: 305.4802134	total: 3.62s	remaining: 4.12s
608:	learn: 303.9302669	total: 3.63s	remaining: 4.12s
609:	learn: 302.2602133	total: 3.63s	remaining: 4.11s
610:	learn: 299.1529377	total: 3.64s	remaining: 4.1s
611:	learn: 295.8362425	total: 3.64s	remaining: 4.09s
612:	learn: 294.2036210	total: 3.65s	remaining: 4.09s
613:	learn: 293.1738592	total: 3.65s	remaining: 4.08s
614:	learn: 292.3895799	total: 3.66s	remaining: 4.07s
615:	learn: 292.3356711	total: 3.66s	remaining: 4.07s
616:	learn: 291.7680157	total: 3.67s	remaining: 4.06s
617:	learn: 290.4479841	total: 3.67s	remaining: 4.05s
618:	learn: 289.7503793	total: 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.5s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


0:	learn: 62892.7474064	total: 579ms	remaining: 12m 32s
1:	learn: 52650.7063821	total: 591ms	remaining: 6m 23s
0:	learn: 60196.2144527	total: 16.5ms	remaining: 21.4s
0:	learn: 63279.2838618	total: 715ms	remaining: 15m 28s
0:	learn: 60505.3824075	total: 30.9ms	remaining: 40.1s
1:	learn: 50367.7217039	total: 738ms	remaining: 7m 58s
1:	learn: 49053.1158226	total: 64ms	remaining: 41.5s
2:	learn: 44267.2455142	total: 641ms	remaining: 4m 37s
2:	learn: 42519.3167210	total: 753ms	remaining: 5m 25s
1:	learn: 49288.1346735	total: 65.9ms	remaining: 42.8s
3:	learn: 39418.9323708	total: 653ms	remaining: 3m 31s
2:	learn: 41012.7798075	total: 76.2ms	remaining: 33s
3:	learn: 36704.4641654	total: 767ms	remaining: 4m 8s
2:	learn: 40624.5352427	total: 87ms	remaining: 37.6s
4:	learn: 35456.0371977	total: 665ms	remaining: 2m 52s
3:	learn: 35988.2036133	total: 113ms	remaining: 36.5s
3:	learn: 35944.9290923	total: 118ms	remaining: 38.3s
5:	learn: 32157.4202640	total: 709ms	remaining: 2m 32s
4:	learn: 33041.8

...

983:	learn: 24.0979375	total: 23.9s	remaining: 7.67s
990:	learn: 17.6319403	total: 23.4s	remaining: 7.29s
1013:	learn: 18.6519871	total: 23.3s	remaining: 6.58s
991:	learn: 17.5257938	total: 23.4s	remaining: 7.26s
966:	learn: 26.1978760	total: 24s	remaining: 8.28s
984:	learn: 23.9858004	total: 23.9s	remaining: 7.65s
967:	learn: 26.0513434	total: 24s	remaining: 8.25s
992:	learn: 17.4119805	total: 23.4s	remaining: 7.23s
1014:	learn: 18.6134225	total: 23.4s	remaining: 6.56s
985:	learn: 23.8693361	total: 23.9s	remaining: 7.62s
968:	learn: 25.7307827	total: 24.1s	remaining: 8.22s
1015:	learn: 18.4397201	total: 23.4s	remaining: 6.54s
986:	learn: 23.5139789	total: 24s	remaining: 7.6s
1016:	learn: 18.4120655	total: 23.4s	remaining: 6.51s
987:	learn: 23.4382095	total: 24s	remaining: 7.57s
993:	learn: 17.3763642	total: 23.4s	remaining: 7.22s
969:	learn: 25.6137224	total: 24.1s	remaining: 8.2s
1017:	learn: 18.3442557	total: 23.4s	remaining: 6.49s
994:	learn: 17.1936835	total: 23.5s	remaining: 7.19

1070:	learn: 11.4305943	total: 25.2s	remaining: 5.38s
1085:	learn: 13.0581425	total: 25.2s	remaining: 4.96s
1058:	learn: 16.3221423	total: 25.7s	remaining: 5.85s
1044:	learn: 17.7836965	total: 25.9s	remaining: 6.31s
1071:	learn: 11.4088781	total: 25.2s	remaining: 5.36s
1086:	learn: 13.0273088	total: 25.2s	remaining: 4.94s
1059:	learn: 16.1192990	total: 25.7s	remaining: 5.83s
1045:	learn: 17.5937868	total: 25.9s	remaining: 6.29s
1060:	learn: 16.1037375	total: 25.8s	remaining: 5.8s
1087:	learn: 12.9135393	total: 25.2s	remaining: 4.92s
1046:	learn: 17.4993993	total: 25.9s	remaining: 6.26s
1072:	learn: 11.2915519	total: 25.2s	remaining: 5.34s
1088:	learn: 12.8734914	total: 25.2s	remaining: 4.89s
1061:	learn: 15.9878905	total: 25.8s	remaining: 5.78s
1089:	learn: 12.7932665	total: 25.3s	remaining: 4.87s
1073:	learn: 11.1911949	total: 25.2s	remaining: 5.31s
1074:	learn: 11.1712916	total: 25.3s	remaining: 5.29s
1047:	learn: 17.4256508	total: 26s	remaining: 6.24s
1062:	learn: 15.8498509	total: 

1134:	learn: 11.0718601	total: 27.5s	remaining: 4s
1163:	learn: 8.4556775	total: 27s	remaining: 3.16s
1117:	learn: 12.3194995	total: 27.7s	remaining: 4.51s
1164:	learn: 8.4119382	total: 27s	remaining: 3.13s
1148:	learn: 7.9361803	total: 27s	remaining: 3.55s
1165:	learn: 8.3742970	total: 27s	remaining: 3.11s
1118:	learn: 12.2492166	total: 27.7s	remaining: 4.48s
1166:	learn: 8.2480507	total: 27.1s	remaining: 3.08s
1135:	learn: 11.0238015	total: 27.5s	remaining: 3.98s
1149:	learn: 7.9052840	total: 27s	remaining: 3.52s
1119:	learn: 12.2433956	total: 27.7s	remaining: 4.46s
1167:	learn: 8.2268525	total: 27.1s	remaining: 3.06s
1136:	learn: 10.9005349	total: 27.6s	remaining: 3.95s
1168:	learn: 8.1468601	total: 27.1s	remaining: 3.04s
1120:	learn: 12.2035669	total: 27.8s	remaining: 4.43s
1137:	learn: 10.7299322	total: 27.6s	remaining: 3.93s
1150:	learn: 7.8759899	total: 27.1s	remaining: 3.5s
1169:	learn: 8.1141877	total: 27.1s	remaining: 3.01s
1121:	learn: 12.1667429	total: 27.8s	remaining: 4.41

Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   51.9s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.4min finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.2min finished


StackingRegressor(cv=5,
                  estimators=[('rf_model',
                               RandomForestRegressor(max_depth=40,
                                                     min_samples_leaf=2,
                                                     min_samples_split=5,
                                                     n_estimators=1600,
                                                     random_state=42)),
                              ('xgb_model',
                               XGBRegressor(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0.4999379628659259,
                                            enable_categorical=False,
                                            gamma=0.0015642079516800683,
                                            gpu_id=...
                                            p

In [18]:
#measure the RMSE on our validation data
predictions = stack.predict(X_valid)
score = mean_squared_error(predictions, y_valid, squared = False)
print('RMSE:', score )

RMSE: 24068.687121300085


In [19]:
#now we will fit our model on the full Kaggle training data
stack.fit(X,y)

1189:	learn: 8.7234232	total: 29.8s	remaining: 2.76s
1240:	learn: 5.4217659	total: 29.1s	remaining: 1.38s
1217:	learn: 7.0302165	total: 29.6s	remaining: 2s
1241:	learn: 5.4207080	total: 29.1s	remaining: 1.36s
1190:	learn: 8.6143939	total: 29.9s	remaining: 2.73s
1225:	learn: 5.1578188	total: 29.1s	remaining: 1.76s
1218:	learn: 7.0105031	total: 29.7s	remaining: 1.97s
1191:	learn: 8.5481112	total: 29.9s	remaining: 2.71s
1242:	learn: 5.3731182	total: 29.1s	remaining: 1.33s
1219:	learn: 6.9600401	total: 29.7s	remaining: 1.95s
1226:	learn: 5.1296007	total: 29.2s	remaining: 1.74s
1227:	learn: 5.0541404	total: 29.2s	remaining: 1.71s
1192:	learn: 8.5309454	total: 29.9s	remaining: 2.68s
1243:	learn: 5.3584987	total: 29.1s	remaining: 1.31s
1228:	learn: 5.0381710	total: 29.2s	remaining: 1.69s
1220:	learn: 6.9372210	total: 29.7s	remaining: 1.92s
1244:	learn: 5.3285456	total: 29.2s	remaining: 1.29s
1229:	learn: 5.0367693	total: 29.2s	remaining: 1.66s
1221:	learn: 6.8989001	total: 29.7s	remaining: 1.

14:	learn: 21571.7997490	total: 119ms	remaining: 10.2s
15:	learn: 21129.3386560	total: 126ms	remaining: 10.1s
16:	learn: 20989.5914352	total: 133ms	remaining: 10s
17:	learn: 20531.7815367	total: 139ms	remaining: 9.93s
18:	learn: 20095.4472899	total: 147ms	remaining: 9.9s
19:	learn: 19882.0337740	total: 154ms	remaining: 9.85s
20:	learn: 19418.6564793	total: 161ms	remaining: 9.83s
21:	learn: 19212.4962945	total: 168ms	remaining: 9.76s
22:	learn: 18891.7513138	total: 176ms	remaining: 9.77s
23:	learn: 18525.3246303	total: 185ms	remaining: 9.85s
24:	learn: 18492.6874622	total: 192ms	remaining: 9.81s
25:	learn: 18169.8521350	total: 200ms	remaining: 9.79s
26:	learn: 18107.7167221	total: 206ms	remaining: 9.73s
27:	learn: 17802.9646780	total: 214ms	remaining: 9.72s
28:	learn: 17453.0863757	total: 222ms	remaining: 9.73s
29:	learn: 17101.1575510	total: 229ms	remaining: 9.7s
30:	learn: 16972.3544126	total: 236ms	remaining: 9.64s
31:	learn: 16668.0005676	total: 242ms	remaining: 9.58s
32:	learn: 164

312:	learn: 2288.0972501	total: 1.94s	remaining: 6.12s
313:	learn: 2287.4080375	total: 1.94s	remaining: 6.11s
314:	learn: 2282.2670661	total: 1.95s	remaining: 6.1s
315:	learn: 2249.8576791	total: 1.96s	remaining: 6.09s
316:	learn: 2249.5806630	total: 1.96s	remaining: 6.08s
317:	learn: 2249.3009519	total: 1.97s	remaining: 6.08s
318:	learn: 2218.9418875	total: 1.97s	remaining: 6.07s
319:	learn: 2203.3122323	total: 1.98s	remaining: 6.06s
320:	learn: 2190.1135498	total: 1.98s	remaining: 6.05s
321:	learn: 2171.2649220	total: 1.99s	remaining: 6.04s
322:	learn: 2164.8309755	total: 2s	remaining: 6.04s
323:	learn: 2149.6231575	total: 2s	remaining: 6.03s
324:	learn: 2136.6124499	total: 2.01s	remaining: 6.02s
325:	learn: 2111.7241374	total: 2.01s	remaining: 6.02s
326:	learn: 2102.9872499	total: 2.02s	remaining: 6.01s
327:	learn: 2093.8628031	total: 2.02s	remaining: 6s
328:	learn: 2083.3733322	total: 2.03s	remaining: 6s
329:	learn: 2060.7059274	total: 2.04s	remaining: 5.99s
330:	learn: 2047.309615

613:	learn: 545.0250789	total: 3.55s	remaining: 3.96s
614:	learn: 544.8083395	total: 3.55s	remaining: 3.96s
615:	learn: 543.7340055	total: 3.56s	remaining: 3.95s
616:	learn: 543.6794043	total: 3.56s	remaining: 3.95s
617:	learn: 540.7607802	total: 3.57s	remaining: 3.94s
618:	learn: 537.5444339	total: 3.58s	remaining: 3.94s
619:	learn: 536.7613368	total: 3.58s	remaining: 3.93s
620:	learn: 535.1168304	total: 3.59s	remaining: 3.92s
621:	learn: 534.1551092	total: 3.59s	remaining: 3.92s
622:	learn: 533.3590744	total: 3.6s	remaining: 3.91s
623:	learn: 529.5649301	total: 3.6s	remaining: 3.9s
624:	learn: 527.0288832	total: 3.61s	remaining: 3.9s
625:	learn: 526.5651882	total: 3.62s	remaining: 3.89s
626:	learn: 525.0781430	total: 3.62s	remaining: 3.88s
627:	learn: 523.1331542	total: 3.63s	remaining: 3.88s
628:	learn: 522.0393225	total: 3.63s	remaining: 3.87s
629:	learn: 517.8536995	total: 3.63s	remaining: 3.87s
630:	learn: 516.1111440	total: 3.64s	remaining: 3.86s
631:	learn: 514.6132324	total: 3

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


...

834:	learn: 108.5350994	total: 11.6s	remaining: 6.45s
835:	learn: 108.3023780	total: 11.6s	remaining: 6.44s
836:	learn: 107.4674491	total: 11.6s	remaining: 6.43s
837:	learn: 106.9194239	total: 11.6s	remaining: 6.42s
838:	learn: 106.6464122	total: 11.7s	remaining: 6.41s
839:	learn: 105.8858922	total: 11.7s	remaining: 6.39s
840:	learn: 105.5345480	total: 11.7s	remaining: 6.38s
841:	learn: 105.1253748	total: 11.7s	remaining: 6.36s
842:	learn: 104.6596069	total: 11.7s	remaining: 6.35s
843:	learn: 104.0935342	total: 11.7s	remaining: 6.34s
844:	learn: 103.3067452	total: 11.7s	remaining: 6.33s
845:	learn: 102.3727626	total: 11.8s	remaining: 6.31s
846:	learn: 102.0255796	total: 11.8s	remaining: 6.3s
847:	learn: 101.7832580	total: 11.8s	remaining: 6.28s
848:	learn: 101.7200822	total: 11.8s	remaining: 6.27s
849:	learn: 101.6056366	total: 11.8s	remaining: 6.25s
850:	learn: 101.1707275	total: 11.8s	remaining: 6.24s
851:	learn: 101.1311339	total: 11.8s	remaining: 6.22s
852:	learn: 100.8997284	total

Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   50.5s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.5min finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.1min finished


StackingRegressor(cv=5,
                  estimators=[('rf_model',
                               RandomForestRegressor(max_depth=40,
                                                     min_samples_leaf=2,
                                                     min_samples_split=5,
                                                     n_estimators=1600,
                                                     random_state=42)),
                              ('xgb_model',
                               XGBRegressor(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0.4999379628659259,
                                            enable_categorical=False,
                                            gamma=0.0015642079516800683,
                                            gpu_id=...
                                            p

917:	learn: 160.3992773	total: 5.07s	remaining: 2.11s
918:	learn: 159.9172876	total: 5.07s	remaining: 2.1s
919:	learn: 159.5786569	total: 5.08s	remaining: 2.1s
920:	learn: 159.5592385	total: 5.08s	remaining: 2.09s
921:	learn: 158.3171377	total: 5.09s	remaining: 2.09s
922:	learn: 157.4196951	total: 5.1s	remaining: 2.08s
923:	learn: 157.2467139	total: 5.1s	remaining: 2.08s
924:	learn: 157.2303749	total: 5.11s	remaining: 2.07s
925:	learn: 156.0807362	total: 5.11s	remaining: 2.06s
926:	learn: 155.5474850	total: 5.12s	remaining: 2.06s
927:	learn: 154.7408316	total: 5.13s	remaining: 2.05s
928:	learn: 154.0826609	total: 5.13s	remaining: 2.05s
929:	learn: 153.9020507	total: 5.14s	remaining: 2.04s
930:	learn: 153.5255643	total: 5.14s	remaining: 2.04s
931:	learn: 153.0102203	total: 5.15s	remaining: 2.03s
932:	learn: 152.9910770	total: 5.15s	remaining: 2.03s
933:	learn: 152.9740822	total: 5.16s	remaining: 2.02s
934:	learn: 152.4761463	total: 5.16s	remaining: 2.02s
935:	learn: 152.3117737	total: 5

1141:	learn: 24.8390209	total: 15.8s	remaining: 2.19s
1142:	learn: 24.7986140	total: 15.8s	remaining: 2.17s
1143:	learn: 24.6542175	total: 15.8s	remaining: 2.16s
1144:	learn: 24.6273651	total: 15.8s	remaining: 2.14s
1145:	learn: 24.4621973	total: 15.9s	remaining: 2.13s
1146:	learn: 24.3517643	total: 15.9s	remaining: 2.12s
1147:	learn: 24.2121560	total: 15.9s	remaining: 2.1s
1148:	learn: 24.0414058	total: 15.9s	remaining: 2.09s
1149:	learn: 24.0385289	total: 15.9s	remaining: 2.08s
1150:	learn: 24.0219251	total: 15.9s	remaining: 2.06s
1151:	learn: 23.8032721	total: 15.9s	remaining: 2.05s
1152:	learn: 23.6621802	total: 16s	remaining: 2.03s
1153:	learn: 23.6399672	total: 16s	remaining: 2.02s
1154:	learn: 23.4847520	total: 16s	remaining: 2s
1155:	learn: 23.4090865	total: 16s	remaining: 1.99s
1156:	learn: 23.3007532	total: 16s	remaining: 1.98s
1157:	learn: 23.2635464	total: 16s	remaining: 1.96s
1158:	learn: 23.2314564	total: 16s	remaining: 1.95s
1159:	learn: 23.1959408	total: 16s	remaining: 

In [20]:
#now we preprocess our competition testing data which is used to make final predictions
test_indices = testing_set.index
testing_set = preprocessor.transform(testing_set)

In [21]:
#make final predictions to submit to competition
final_pred = stack.predict(testing_set)
#output predictions as df for Kaggle competition
final_pred = pd.DataFrame({'Id': test_indices,
                       'SalePrice': final_pred})
final_pred.to_csv(data_path + 'predictions/ensemble_submission.csv', index=False)

In [22]:
#finally we save our ensemble model
with open(ensemble_path + f'ensemble_model.pickle', 'wb') as f:
        pickle.dump(stack, f)

Our final ensemble model receives a RMSE score of 13,969 on the competition data and scores in the top 1%!